In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR

from torch.utils.data import DataLoader
from BookDataset import getBookDataset, BookDataset
from embeddings import EmbeddingSummation

In [2]:
train, val, test, featureSizes = getBookDataset('../Data_Train.xlsx', '../Data_Test.xlsx')

In [3]:
class BookPrice(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        outDims = lambda x: int(x**.25) + 1
        self.outDims = outDims
        
        # embeddings for categorical variables: Author, Genre, BookCategory
        self.AuthorEmbedding = nn.Embedding(config.Author,  outDims(config.Author))
        self.GenreEmbedding  = nn.Embedding(config.Genre,  outDims(config.Genre))
        self.BookCategoryEmbedding  = nn.Embedding(config.BookCategory,  outDims(config.BookCategory))
        categoricalUnits = 5 * outDims(config.Author) + outDims(config.Genre) + outDims(config.BookCategory)
        # ------
        
        
        # embeddings for text features Title, Synopsis
        self.TitleEmbedding    = EmbeddingSummation()
        self.SynopsisEmbedding = EmbeddingSummation()
        textUnits = 128 * 2
        #-------
        
        # Ratings, Reviews and Edition
        numericUnits = 3
        #-------

        total = categoricalUnits + textUnits + numericUnits
        
        self.Dense = nn.Sequential(
            nn.Linear(total, 64), nn.LayerNorm(64), nn.Tanh(), nn.Dropout(.2),
            nn.Linear(64, 32), nn.LayerNorm(32), nn.Tanh(), nn.Dropout(.2),
            nn.Linear(32, 1)
        )

    def forward(self, batch:dict) -> torch.Tensor:
        authors = torch.stack([ batch[f'author_{i}'] for i in range(5) ]).T
        authDim = 5 * self.outDims(self.config.Author)
        categoricals = [
            self.AuthorEmbedding(authors).reshape(-1, authDim),
            self.GenreEmbedding(batch['Genre']),
            self.BookCategoryEmbedding(batch['BookCategory'])
        ]
        
        text = [
            self.TitleEmbedding(batch['Title']),
            self.SynopsisEmbedding(batch['Synopsis'])
        ]
        
        numericals  = [torch.stack([
            batch['Ratings'],
            batch['Reviews'],
            batch['Edition']
        ]).T]
        
        inputs = torch.cat(categoricals + text + numericals, dim=-1)
        
        logits = self.Dense(inputs)
        return logits
    
    
    def training_step(self, batch:dict, batchIdx:int)->torch.Tensor:
        logits = self(batch)

        loss = F.mse_loss(logits, batch['Price'].reshape(-1, 1))
        self.log('loss', loss)

        return loss

    def validation_step(self, batch:dict, batchIdx:int):
        logits = self(batch)
        loss = F.mse_loss(logits, batch['Price'].reshape(-1, 1))
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=1e-3)
        scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.99 * epoch)
        return [optimizer], [scheduler]

    
model = BookPrice(featureSizes)
x = sum(p.numel() for p in model.parameters())
print(f'BookPrice model has {x:,} parameters')

BookPrice model has 7,836,458 parameters


In [4]:
stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')
ckpt = ModelCheckpoint(dirpath='checkpoints', filename='{epoch}-{val_loss:.5f}',
                       monitor='val_loss', mode='min',
                      save_weights_only=True, verbose=True)

trainer = pl.Trainer(gpus = 1, callbacks=[stopping, ckpt])

loaders = []
for i, data in enumerate([train, val, test]):
    shuffle = i==0
    loaders.append( DataLoader(BookDataset(data), batch_size=16, shuffle=shuffle ) )

trainer.fit(model, loaders[0], loaders[1])

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type               | Params
-------------------------------------------------------------
0 | AuthorEmbedding       | Embedding          | 2.6 K 
1 | GenreEmbedding        | Embedding          | 804   
2 | BookCategoryEmbedding | Embedding          | 24    
3 | TitleEmbedding        | EmbeddingSummation | 3.9 M 
4 | SynopsisEmbedding     | EmbeddingSummation | 3.9 M 
5 | Dense                 | Sequential         | 20.9 K
-------------------------------------------------------------
7.8 M     Trainable params
0         Non-trainable params
7.8 M     Total params


Epoch 0, global step 311: val_loss reached 45.44375 (best 45.44375), saving model to "C:\Users\Deepak H R\Desktop\data\BookPrice\albert\chekpoints\epoch=0-val_loss=45.44375.ckpt" as top 1


Epoch 1, global step 623: val_loss reached 0.61094 (best 0.61094), saving model to "C:\Users\Deepak H R\Desktop\data\BookPrice\albert\chekpoints\epoch=1-val_loss=0.61094.ckpt" as top 1


Epoch 2, global step 935: val_loss reached 0.60454 (best 0.60454), saving model to "C:\Users\Deepak H R\Desktop\data\BookPrice\albert\chekpoints\epoch=2-val_loss=0.60454.ckpt" as top 1


Epoch 3, global step 1247: val_loss reached 0.57148 (best 0.57148), saving model to "C:\Users\Deepak H R\Desktop\data\BookPrice\albert\chekpoints\epoch=3-val_loss=0.57148.ckpt" as top 1


...

Epoch 49, step 15599: val_loss was not in top 1


Epoch 50, step 15911: val_loss was not in top 1


Epoch 51, step 16223: val_loss was not in top 1


Epoch 52, step 16535: val_loss was not in top 1


1

In [5]:
print("Saving model from best ckpt", ckpt.best_model_path)
model = BookPrice.load_from_checkpoint(ckpt.best_model_path, config=featureSizes).eval().cuda()
results = []
with torch.no_grad():
    for batch in loaders[2]:
        for key in batch:
            if key not in 'Title Synopsis'.split():
                batch[key] = batch[key].to(model.device)
        preds = model(batch)
        results.append(preds)
results = torch.cat(results).cpu().numpy().reshape(-1)
submission = pd.DataFrame({
    'Price': np.exp(results) - 1
})
print("Saving submission ", submission.shape)
submission.to_excel('submission.xlsx', index=False)

Saving model from best ckpt C:\Users\Deepak H R\Desktop\data\BookPrice\albert\chekpoints\epoch=42-val_loss=0.30309.ckpt
Saving submission  (1560, 1)
